# How to use XtracTree?
In this notebook, we demonstrate the usage of XtracTree for a Decision Tree Classifier. Parameter "exp" can be adjusted to do the experiments with a Random Forest Classifier.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (classification_report, 
                             confusion_matrix, 
                             roc_curve, 
                             auc)
from sklearn.utils import Bunch
import pickle as pkl

# pandas options
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

# XtracTree Class Definition
Here we pasted the XtracTree class in the notebook for the sake of the demo. It is possible to save the class in a separate module.

In [ ]:
class XtracTree:
    def __init__(self, estimator, x_train, x_test, 
                 sample_id=None, sample_ids=None, out=None):
        self.estimator = estimator
        self.x_train = x_train
        self.x_test = x_test
        self.sample_id = sample_id
        self.sample_ids = sample_ids
        self.out = out


    def build_dtree_rules(estimator, n_trees, x_train, l2w):
        """Extract decision rules with splitting thresholds and probabilities.
        """
        # we can get useful information about the tree structure
        # using estimator.tree.__getstate__()['nodes']
        # or import sklearn, help(sklearn.tree._tree.Tree)
        n_nodes = estimator.tree_.node_count
        children_left = estimator.tree_.children_left
        children_right = estimator.tree_.children_right
        feature = estimator.tree_.feature
        threshold = estimator.tree_.threshold
        n_node_samples = estimator.tree_.n_node_samples
        value = estimator.tree_.value 
        weighted_n_node_samples = estimator.tree_.weighted_n_node_samples

        node_depth = np.zeros(shape = n_nodes, dtype = np.int64)
        is_leaves = np.zeros(shape = n_nodes, dtype = bool)
        stack = [(0, -1)] # seed is the root node id and its parent depth

        while len(stack) > 0:
            node_id, parent_depth = stack.pop()
            node_depth[node_id] = parent_depth + 1

            # if we have a test node
            if (children_left[node_id] != children_right[node_id]):
                stack.append((children_left[node_id], parent_depth + 1))
                stack.append((children_right[node_id], parent_depth + 1))
            else:
                is_leaves[node_id] = True

        for i in range(n_nodes):
            if is_leaves[i]:
                # irow = "%sif state == %s:\n%sprint(state)\n%sreturn %s\n" % ((
                irow = "        if state == %s: return %s\n" % (
                    i, ((value[i][0,1] / weighted_n_node_samples[i]) / n_trees)
                    )
            else:
                irow = "        if state == %s: state = (%s if x['%s'] <= %s else %s)\n" % (
                    i, 
                    children_left[i],
                    x_train.columns[feature[i]],
                    threshold[i],
                    children_right[i]
                )
            l2w.append(irow)
        return l2w


    def build_model(self):
        """Build the global architecture of the bagging algorithm.
        It is designed such that it is an executable if self.out is 
        saved as .py file. 
        """
        # we write the first line of the file
        # we store the content of the file to be written in l2w
        l2w = ["import numpy as np\n"]
        l2w.append("\n")
        l2w.append("def estimator_tree(x, num_tree):\n")
        l2w.append("    if num_tree == %s:\n" % 0)
        l2w.append("        state = %s\n" % 0)

        if 'RandomForestClassifier' in str(type(self.estimator)):
            n_trees = len(self.estimator.estimators_)
            for n in range(n_trees):
                if n == 0:
                    l2w = XtracTree.build_dtree_rules(
                        self.estimator.estimators_[n], 
                        n_trees, self.x_train, l2w)
                else:
                    l2w.append("    elif num_tree == %s:\n" % n)
                    l2w.append("        state = %s\n" % 0)
                    l2w = XtracTree.build_dtree_rules(
                        self.estimator.estimators_[n], 
                        n_trees, self.x_train, l2w)
        else:
            # the estimator is a decision tree
            # we can pass it directly to build_tree_rules
            n_trees = 1
            l2w = XtracTree.build_dtree_rules(
                        self.estimator, 
                        n_trees, self.x_train, l2w)
        
        # we write at the bottom the predict rule for the fixed model
        l2w.append("\n\ndef estimator_predict(x):\n")

        # we initialize the proba values at 0
        l2w.append("    predict = 0.0\n")
        l2w.append("    for i in range(%s):\n" % n_trees)
        l2w.append("        predict += estimator_tree(x, i)\n")
        l2w.append("    return predict\n")

        if self.out is not None:
            with open(self.out, 'w') as the_file:
                the_file.write("".join(l2w))
                the_file.close()
        else:
            print(l2w)
        
        return None


    def display_rule_per_estimator(estimator, X_test, sample_id, l2r):
        """Display the decision path per tree contained in the estimator for 1 sample.
        """
        n_nodes = estimator.tree_.node_count
        children_left = estimator.tree_.children_left
        children_right = estimator.tree_.children_right
        feature = estimator.tree_.feature
        threshold = estimator.tree_.threshold
        node_indicator = estimator.decision_path(X_test)
        
        # we have the leaves ids reached by each sample.
        leave_id = estimator.apply(X_test)

        node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                                                        node_indicator.indptr[sample_id + 1]]
        
        for node_id in node_index:
            if leave_id[sample_id] == node_id:
                continue

            if (X_test.iloc[sample_id, feature[node_id]] <= threshold[node_id]):
                threshold_sign = "<="
            else:
                threshold_sign = ">"

            print("decision node %s: %s (=%s) %s %s"
                        % (node_id,
                                X_test.columns[feature[node_id]],
                                X_test.iloc[sample_id, feature[node_id]],
                                threshold_sign,
                                np.round(threshold[node_id],4)))
            l2r.append(
                    [X_test.columns[feature[node_id]], 
                    X_test.iloc[sample_id, feature[node_id]],
                    threshold[node_id]]
            )
        return l2r


    def display_rule_per_estimator_sample_ids(estimator, X_test, sample_ids):
        """Display the decision path per tree contained in the estimator for a group of samples.
        """
        n_nodes = estimator.tree_.node_count
        children_left = estimator.tree_.children_left
        children_right = estimator.tree_.children_right
        feature = estimator.tree_.feature
        threshold = estimator.tree_.threshold
        node_indicator = estimator.decision_path(X_test)
        
        # we have the leaves ids reached by each sample.
        leave_id = estimator.apply(X_test)

        common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                                        len(sample_ids))

        common_node_id = np.arange(n_nodes)[common_nodes]

        print("Shared nodes %s," % (common_node_id),
                    "Shared features %s," % (
                        X_test.columns[feature[common_node_id]].values), 
                    "Shared threshold %s" % (threshold[common_node_id]
                    )
        )
        print("Values:\n%s" % (X_test.iloc[sample_ids, feature[common_node_id]].values))

        return None


    def sample_rules(self):
        """Display decision path on demand if sample_id or sample_ids is not None.
        """
        l2r = []

        if self.sample_id is not None:
            print("Rules to predict sample %s" % self.sample_id)
            if 'RandomForestClassifier' in str(type(self.estimator)):
                n_trees = len(self.estimator.estimators_)
                for n in range(n_trees):
                    print("\nRules for tree %s" % n)
                    l2r = XtracTree.display_rule_per_estimator(
                            self.estimator.estimators_[n],
                            self.x_test, self.sample_id, l2r
                    )
            else:
                # the estimator is a decision tree
                # we can pass it directly to display_rule_per_estimator
                l2r = XtracTree.display_rule_per_estimator(
                            self.estimator, self.x_test, 
                            self.sample_id, l2r
                )
            
            # we convert l2r as a dataframe
            l2r = pd.DataFrame(l2r, 
                               columns=['Features', 
                                        'Value Sample %s ' % self.sample_id, 
                                        'Threshold']
            )
            
        if self.sample_ids is not None:
            print("\n\nRules to predict samples %s" % self.sample_ids)
            if 'RandomForestClassifier' in str(type(self.estimator)):
                n_trees = len(self.estimator.estimators_)
                for n in range(n_trees):
                    print("\nRules for tree %s" % n)
                    XtracTree.display_rule_per_estimator_sample_ids(
                        self.estimator.estimators_[n],
                        self.x_test, self.sample_ids
                    )
            else:
                # the estimator is a decision tree
                # we can pass it directly to display_rule_per_estimator
                XtracTree.display_rule_per_estimator_sample_ids(
                    self.estimator, self.x_test, self.sample_ids
                )

        if len(l2r): return l2r

# Data Processing
Data was pre-processed and save in .pkl file called loanForExp.pkl.

In [ ]:
# we import the pkl file containing the data
loanForExp = pkl.load(open('loanForExp.pkl','rb'), encoding='latin1')
X_train, X_test, y_train, y_test = train_test_split(
    loanForExp.data,
    loanForExp.target,
    test_size=0.3,
    shuffle=True
)

# Estimator Definition
in this demo, we illustrate XtracTree with a Decision Tree Classifier. By changing exp=1, one can test the notebook for a Random Forest Classifier.

In [ ]:
exp = 2
if exp == 1:
    estimator = (
        RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
    )
else:
    estimator = (
        DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, 
                      criterion='gini', max_depth=10, max_features=50, 
                      max_leaf_nodes=30, min_impurity_decrease=0.0, 
                      min_impurity_split=None, min_samples_leaf=1, 
                      min_samples_split=2, min_weight_fraction_leaf=0.0, 
                      presort='deprecated', random_state=0, splitter='best')
    )

estimator.fit(X_train, y_train) # model fit

# we convert the features importance of the classifier to a df
d = {'Features': X_train.columns, 
     'Feat Imp': estimator.feature_importances_
}
estimatorFeatimportance = pd.DataFrame(d).sort_values(
    by='Feat Imp', ascending=False
)

# XtracTree
XtracTree is first initialized with the trained estimator, the train set and the test set. Then, the desicion rules of the estimator are extracted and saved in the file estimator_decision_rules.py.

In [ ]:
p = XtracTree(estimator, X_train, X_test, out='estimator_decision_rules.py')
p.build_model()
#
# output the contents of estimator_decision_rules.py
file1 = open('estimator_decision_rules.py', 'r')
Lines = file1.readlines()
count = 0
print("!!! Contents written in estimator_decision_rules.py by XtracTree !!!\n\n")
for line in Lines:
    count += 1
    print("{} {}".format(count, line))

!!! Contents written in estimator_decision_rules.py by XtracTree !!!


1 import numpy as np

2 

3 def estimator_tree(x, num_tree):

4     if num_tree == 0:

5         state = 0

6         if state == 0: state = (1 if x['total_rec_prncp'] <= 998.0950012207031 else 2)

7         if state == 1: return 1.0

8         if state == 2: state = (3 if x['out_prncp'] <= 13385.35546875 else 4)

9         if state == 3: state = (5 if x['total_rec_prncp'] <= 1191.5700073242188 else 6)

10         if state == 4: return 1.0

11         if state == 5: state = (7 if x['loan_amnt'] <= 5550.0 else 8)

12         if state == 6: state = (9 if x['total_rec_int'] <= 0.16500000655651093 else 10)

13         if state == 7: return 0.0

14         if state == 8: return 1.0

15         if state == 9: state = (11 if x['il_util'] <= 88.5 else 12)

16         if state == 10: state = (19 if x['last_pymnt_amnt'] <= 1347.9049682617188 else 20)

17         if state == 11: state = (13 if x['all_util'] <= 92.0 else 14)

1

Using the custom funtion estimator_predict(*args) generated in estimator_decision_rules.py, XtracTree can replicate exactly the predictive performance of the trained ML estimator simply using the rules-based model.

In [ ]:
from estimator_decision_rules import estimator_predict

sample_ids = [0,1,2,3,4,5]
res_from_parser = np.zeros((len(sample_ids)))
for n in range(len(sample_ids)):
  sample_id = sample_ids[n]
  sample_proba = estimator_predict(X_test.iloc[sample_id, :])
  res_from_parser[n] = sample_proba

d = {"sample": sample_ids, 
     "Proba from XtracTree": res_from_parser, 
     "Proba from DT classifier": estimator.predict_proba(X_test)[:, 1][sample_ids]}
pd.DataFrame(d)

,sample,Proba from XtracTree,Proba from DT classifier
0,0,0.0,0.0
1,1,0.0,0.0
2,2,0.0,0.0
3,3,0.0,0.0
4,4,0.0,0.0
5,5,0.0,0.0


In [ ]:
sample_ids = np.arange(0, len(X_test[:15000]))
res_from_parser = np.zeros((len(sample_ids)))
for n in range(len(sample_ids)):
  sample_id = sample_ids[n]
  sample_proba = estimator_predict(X_test.iloc[sample_id, :])
  res_from_parser[n] = sample_proba

d = {"sample": sample_ids, 
     "Proba from XtracTree": res_from_parser, 
     "Proba from DT classifier": estimator.predict_proba(X_test)[:, 1][sample_ids]}
pd.DataFrame(d).describe()

,sample,Proba from XtracTree,Proba from DT classifier
count,3000.000000,3000.000000,3000.000000
mean,1499.500000,0.049471,0.049471
std,866.169729,0.216586,0.216586
min,0.000000,0.000000,0.000000
25%,749.750000,0.000000,0.000000
50%,1499.500000,0.000000,0.000000
75%,2249.250000,0.000000,0.000000
max,2999.000000,1.000000,1.000000


# Decision path on demand
With XtracTree, one can highlight the complex decision process that led to the predictions. In other words, the decision path can be outputed for one sample or for a group of samples.

In [ ]:
p = XtracTree(estimator, X_train, X_test, sample_id=6, sample_ids=[0,1,2])
df_rules = p.sample_rules()

Rules to predict sample 6
decision node 0: total_rec_prncp (=10000.0) > 998.095
decision node 2: out_prncp (=0.0) <= 13385.3555
decision node 3: total_rec_prncp (=10000.0) > 1191.57
decision node 6: total_rec_int (=230.29) > 0.165
decision node 10: last_pymnt_amnt (=9898.17) > 1347.905


Rules to predict samples [0, 1, 2]
Shared nodes [ 0  2  3  6 10 20], Shared features ['total_rec_prncp' 'out_prncp' 'total_rec_prncp' 'total_rec_int'
 'last_pymnt_amnt' 'total_bc_limit'], Shared threshold [ 9.98095001e+02  1.33853555e+04  1.19157001e+03  1.65000007e-01
  1.34790497e+03 -2.00000000e+00]
Values:
[[ 8000.       0.    8000.     637.5   7473.88 18100.  ]
 [15000.       0.   15000.     587.33 14075.93 11500.  ]
 [ 4500.       0.    4500.     130.69  4635.67 17000.  ]]
